# Current

## Create test example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import string, random, toolz, pandas, importlib, sys, os

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    191729.000000
mean        259.784759
std         258.718918
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3139.000000
dtype: float64

## Run current version

In [5]:
!git stash
!git checkout main
#importlib.reload(dimod)

Saved working directory and index state WIP on binary-polynomial: 727ca81 Merge branch 'dwavesystems:main' into binary-polynomial
Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [6]:
from dimod import BinaryPolynomial as BPold

In [7]:
BPold.__init__??

Signature: BPold.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        self._terms = terms = {}
        for term, bias in poly:

            fsterm = asfrozenset(term)

            # when SPIN-valued, s^2 == 1, so we need to handle that case
            # in BINARY, x^2 == x
            if len(fsterm) < len(term) and vartype is Vartype.SPIN:
                new = set()
                term = tuple(term)  # make sure it has .count
                for v in fsterm:
                    if term.count(v) % 2:
                        new.add(v)
                fsterm = frozenset(new)

            if fsterm in terms:
                terms[fsterm] += bias
            else:
                terms[fsterm] = bias

        self.vartype = 

In [8]:
%%prun -s cumtime -l 20
bp = BPold(terms, 'SPIN')

         9692045 function calls (9691965 primitive calls) in 28.029 seconds

   Ordered by: cumulative time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   28.029   28.029 {built-in method builtins.exec}
        1    0.000    0.000   28.029   28.029 <string>:1(<module>)
        1    0.000    0.000   28.029   28.029 decorators.py:289(new_f)
        1    2.595    2.595   28.029   28.029 polynomial.py:89(__init__)
  5469142   24.517    0.000   24.517    0.000 {method 'count' of 'tuple' objects}
   191729    0.579    0.000    0.605    0.000 polynomial.py:32(asfrozenset)
  3455784    0.274    0.000    0.274    0.000 {method 'add' of 'set' objects}
   383460    0.038    0.000    0.038    0.000 {built-in method builtins.len}
   191739    0.026    0.000    0.026    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
    

In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,181267.000000,181267.000000
mean,18.937131,1.057716
std,7.558504,8.691843
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,40.000000,3663.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')
BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')

486 ms ± 5.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')
BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')

774 ms ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [13]:
len(BPold({term: 1}, 'SPIN').to_binary().to_spin())

4096

In [14]:
len(BPold({term: 1}, 'BINARY').to_spin().to_binary())

4096

# Exit interpreter
I just can't get module reloading to work properly here ...

In [15]:
print('BYE')
exit()
raise SystemExit
print('HI')

BYE


SystemExit: 

/home/daniel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Proposed

## Create test example

In [1]:
import string, random, toolz, pandas, importlib, sys, os, cytoolz, toolz

In [2]:
import itertools as it

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    192653.000000
mean        258.533986
std         257.308604
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        2894.000000
dtype: float64

## Run proposed version

In [5]:
!mv polynomial-perf.ipynb{,.bak}
!git checkout binary-polynomial
!git stash pop
#!touch -c ./dimod/higherorder/polynomial.py
#importlib.reload(dimod)

M	dimod/higherorder/polynomial.py
D	polynomial-perf.ipynb
Already on 'binary-polynomial'
Your branch is up to date with 'origin/binary-polynomial'.
No stash entries found.


In [6]:
from dimod import BinaryPolynomial as BPnew

In [7]:
BPnew.__init__??

Signature: BPnew.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms

        terms = defaultdict(int)
        #for term, bias in poly:
        for term, bias in filter(tl.second, poly):
            terms[freeze_term(term, vartype)] += bias

        self._terms = tl.valfilter(bool, terms)
        self.vartype = vartype
File:      ~/dimod/dimod/higherorder/polynomial.py
Type:      function


In [8]:
%%prun -s cumtime -l 20 
bp = BPnew(terms, 'SPIN')

         4240433 function calls (4240353 primitive calls) in 3.260 seconds

   Ordered by: cumulative time
   List reduced from 46 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.260    3.260 {built-in method builtins.exec}
        1    0.000    0.000    3.260    3.260 <string>:1(<module>)
        1    0.003    0.003    3.260    3.260 decorators.py:289(new_f)
        1    0.223    0.223    3.257    3.257 polynomial.py:108(__init__)
   192653    2.586    0.000    3.034    0.000 polynomial.py:40(freeze_term)
  3662271    0.419    0.000    0.419    0.000 polynomial.py:47(<genexpr>)
   192663    0.016    0.000    0.016    0.000 {built-in method builtins.isinstance}
   192654    0.013    0.000    0.013    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck

In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,181958.000000,181958.000000
mean,18.937804,1.058777
std,7.557633,8.780509
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,42.000000,3707.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')
BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')

789 ms ± 66.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')
BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')

558 ms ± 70.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [13]:
%timeit zz = list(cytoolz.concat(bp.keys()))

88.8 ms ± 460 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit zz = list(it.chain.from_iterable(bp.keys()))

89.3 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
BPnew({term: 1}, 'BINARY').to_spin().to_binary()

BinaryPolynomial({frozenset({'f', 'k', 'c', 'e', 'b', 'i', 'a', 'h', 'j', 'd', 'l', 'g'}): 1.0}, 'BINARY')

In [16]:
BPnew({term: 1}, 'SPIN').to_binary().to_spin()

BinaryPolynomial({frozenset({'f', 'k', 'c', 'e', 'b', 'i', 'a', 'h', 'j', 'd', 'l', 'g'}): 1.0}, 'SPIN')

In [17]:
BPnew({term: 1}, 'BINARY').to_spin() * BPnew({term: 1}, 'BINARY').to_spin()

ValueError: not enough values to unpack (expected 2, got 0)